In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(r'./countries/countries.db')
def rsq(sql):
    return pd.read_sql(sql, conn)

In [2]:
rsq("""SELECT 
    c1.name AS city,
    code,
    c2.name AS country,
    region,
    city_proper_pop
FROM cities AS c1
-- Perform an inner join with cities as c1 and countries as c2 on country code
inner join countries as c2 
on  c1.country_code =c2.code 
ORDER BY code DESC;""").head()

,city,code,country,region,city_proper_pop
0,Harare,ZWE,Zimbabwe,Eastern Africa,1606000
1,Lusaka,ZMB,Zambia,Eastern Africa,1742979
2,Cape Town,ZAF,South Africa,Southern Africa,3740026
3,Durban,ZAF,South Africa,Southern Africa,3442361
4,Ekurhuleni,ZAF,South Africa,Southern Africa,3178470


In [3]:
rsq("""SELECT 
	c1.name AS city, 
    code, 
    c2.name AS country,
    region, 
    city_proper_pop
FROM cities AS c1
-- Join right table (with alias)
left join countries as c2
ON c1.country_code = c2.code
ORDER BY code DESC;""").head()

,city,code,country,region,city_proper_pop
0,Harare,ZWE,Zimbabwe,Eastern Africa,1606000
1,Lusaka,ZMB,Zambia,Eastern Africa,1742979
2,Cape Town,ZAF,South Africa,Southern Africa,3740026
3,Durban,ZAF,South Africa,Southern Africa,3442361
4,Ekurhuleni,ZAF,South Africa,Southern Africa,3178470


In [4]:
rsq("""SELECT name, region, gdp_percapita
FROM countries AS c
LEFT JOIN economies AS e
ON c.code = e.code
WHERE year = 2010;""").head()

,name,region,gdp_percapita
0,Afghanistan,Southern and Central Asia,539.667
1,Angola,Central Africa,3599.270
2,Albania,Southern Europe,4098.130
3,United Arab Emirates,Middle East,34628.630
4,Argentina,South America,10412.950


In [5]:
rsq("""-- Select region, and average gdp_percapita as avg_gdp
Select region, avg(gdp_percapita) as avg_gdp
FROM countries AS c
LEFT JOIN economies AS e
USING(code)
WHERE year = 2010
-- Group by region
GROUP BY region;""").head()

,region,avg_gdp
0,Australia and New Zealand,44792.385000
1,Baltic Countries,12631.030000
2,British Islands,43588.330000
3,Caribbean,11413.339462
4,Central Africa,4797.239889


In [6]:
rsq("""SELECT region, AVG(gdp_percapita) AS avg_gdp
FROM countries AS c
LEFT JOIN economies AS e
USING(code)
WHERE year = 2010
GROUP BY region
-- Order by descending avg_gdp
Order BY avg_gdp desc
-- Return only first 10 records
limit 10""")

,region,avg_gdp
0,Western Europe,58130.962857
1,Nordic Countries,57073.998000
2,North America,47911.510000
3,Australia and New Zealand,44792.385000
4,British Islands,43588.330000
5,Eastern Asia,24962.808000
6,Southern Europe,22926.410909
7,Middle East,18204.641765
8,Baltic Countries,12631.030000
9,Caribbean,11413.339462


In [9]:
rsq("""-- Modify this query to use RIGHT JOIN instead of LEFT JOIN
SELECT countries.name AS country, languages.name AS language, percent
FROM languages
RIGHT JOIN countries
USING(code)
ORDER BY language;""").head()

DatabaseError: Execution failed on sql '-- Modify this query to use RIGHT JOIN instead of LEFT JOIN
SELECT countries.name AS country, languages.name AS language, percent
FROM languages
RIGHT JOIN countries
USING(code)
ORDER BY language;': RIGHT and FULL OUTER JOINs are not currently supported

In [10]:
rsq("""-- Modify this query to use RIGHT JOIN instead of LEFT JOIN
SELECT countries.name AS country, languages.name AS language, percent
FROM countries
LEFT JOIN languages
USING(code)
ORDER BY language;""").head()


,country,language,percent
0,"Virgin Islands, British",None,NaN
1,Fiji Islands,None,NaN
2,Cape Verde,None,NaN
3,Macao,None,NaN
4,Czech Republic,None,NaN


In [13]:
rsq("""SELECT name AS country, code, region, basic_unit
FROM countries
-- Join to currencies
FULL JOIN currencies
USING (code)
-- Where region is North America or name is null
WHERE region = 'North America' OR name IS NULL
ORDER BY region;""").head()

DatabaseError: Execution failed on sql 'SELECT name AS country, code, region, basic_unit
FROM countries
-- Join to currencies
FULL JOIN currencies
USING (code)
-- Where region is North America or name is null
WHERE region = 'North America' OR name IS NULL
ORDER BY region;': RIGHT and FULL OUTER JOINs are not currently supported

In [20]:
rsq("""SELECT name AS country, code, region, basic_unit
FROM countries
-- Join to currencies
LEFT JOIN currencies
USING (code)
-- Where region is North America or name is null
WHERE region = 'North America' OR name IS NULL
UNION
SELECT name AS country, code, region, basic_unit
FROM currencies
-- Join to countries
LEFT JOIN countries
USING (code)
-- Where region is North America or name is null
WHERE region = 'North America' OR name IS NULL
ORDER BY country desc;""").shape


(18, 4)

In [19]:
rsq("""SELECT name AS country, code, region, basic_unit
FROM countries
-- Join to currencies
left join currencies 
USING (code)
WHERE region = 'North America' 
	OR name IS NULL
ORDER BY region;""").shape

(4, 4)

In [18]:
rsq("""SELECT name AS country, code, region, basic_unit
FROM countries
-- Join to currencies
inner join currencies 
USING (code)
WHERE region = 'North America' 
	OR name IS NULL
ORDER BY region;""").shape

(3, 4)

In [21]:
rsq("""SELECT 
	c1.name AS country, 
    region, 
    l.name AS language,
    basic_unit, 
    frac_unit
FROM countries as c1 
-- Full join with languages (alias as l)
FULL JOIN languages as l ON c1.code = l.code
-- Full join with currencies (alias as c2)
FULL JOIN currencies as c2 ON c1.code = c2.code
WHERE region LIKE 'M%esia';""").head()

DatabaseError: Execution failed on sql 'SELECT 
	c1.name AS country, 
    region, 
    l.name AS language,
    basic_unit, 
    frac_unit
FROM countries as c1 
-- Full join with languages (alias as l)
FULL JOIN languages as l ON c1.code = l.code
-- Full join with currencies (alias as c2)
FULL JOIN currencies as c2 ON c1.code = c2.code
WHERE region LIKE 'M%esia';': RIGHT and FULL OUTER JOINs are not currently supported

In [24]:
rsq("""SELECT 
	c1.name AS country, 
    region, 
    l.name AS language,
    basic_unit, 
    frac_unit
FROM countries as c1 
-- Left join with languages (alias as l)
LEFT JOIN languages as l ON c1.code = l.code
-- Left join with currencies (alias as c2)
LEFT JOIN currencies as c2 ON c1.code = c2.code
WHERE region LIKE 'M%esia';""").head()


,country,region,language,basic_unit,frac_unit
0,Fiji Islands,Melanesia,None,None,None
1,Guam,Micronesia,Asian,None,None
2,Guam,Micronesia,Chamorro,None,None
3,Guam,Micronesia,English,None,None
4,Guam,Micronesia,Filipino,None,None


In [26]:
rsq("""SELECT c.name AS country, l.name AS language 
FROM countries AS c 
-- Inner join countries as c with languages as l on code
INNER JOIN languages AS l ON c.code = l.code 
WHERE c.code IN ('PAK','IND') AND l.code IN ('PAK','IND');""").head()

,country,language
0,India,Assamese
1,India,Bengali
2,India,Gujarati
3,India,Hindi
4,India,Kannada


In [30]:
rsq("""SELECT c.name AS country, l.name AS language
FROM countries AS c        
-- Perform a cross join to languages (alias as l)
CROSS JOIN languages AS l
WHERE c.code in ('PAK','IND')
	AND l.code in ('PAK','IND');""")

,country,language
0,India,Hindi
1,India,Bengali
2,India,Telugu
3,India,Marathi
4,India,Tamil
5,India,Urdu
6,India,Gujarati
7,India,Kannada
8,India,Malayalam
9,India,Oriya


In [31]:
rsq("""SELECT 
    c.name AS country,
    region,
    life_expectancy AS life_exp
FROM 
-- -- Join to populations (alias as p) using an appropriate join
    countries AS c
    JOIN populations AS p ON c.code = p.country_code
-- -- Filter for only results in the year 2010
WHERE
    year = 2010
ORDER BY 
    life_exp
-- -- Limit to five records
LIMIT 5;""")

,country,region,life_exp
0,Andorra,Southern Europe,None
1,American Samoa,Polynesia,None
2,Cayman Islands,Caribbean,None
3,Dominica,Caribbean,None
4,Gibraltar,Southern Europe,None


In [33]:
rsq("""-- Select aliased fields from populations as p1
SELECT p1.country_code, p1.size AS size2010, p2.size AS size2015 
FROM populations p1 
INNER JOIN populations p2 ON p1.country_code = p2.country_code;
-- Join populations as p1 to itself, alias as p2, on country code
""").head()

,country_code,size2010,size2015
0,ABW,101597.0,101597.0
1,ABW,101597.0,103889.0
2,ABW,103889.0,101597.0
3,ABW,103889.0,103889.0
4,AFG,27962207.0,27962207.0


In [35]:
rsq("""SELECT 
	p1.country_code, 
    p1.size AS size2010, 
    p2.size AS size2015
FROM populations AS p1
INNER JOIN populations AS p2
ON p1.country_code = p2.country_code
WHERE p1.year = 2010
-- Filter such that p1.year is always five years before p2.year
    AND p2.year = (p1.year + 5)""").head()

,country_code,size2010,size2015
0,ABW,101597.0,103889.0
1,AFG,27962207.0,32526562.0
2,AGO,21219954.0,25021974.0
3,ALB,2913021.0,2889167.0
4,AND,84419.0,70473.0
